In [1]:
"""
Hyperparameter Tuning & Validation Suite
=========================================

Goal: Find optimal model configuration and validate success criteria

Success Criteria:
1. Positive R² score (model better than mean prediction)
2. Trajectory predictions qualitatively match ground truth
3. Model generalizes across strains (leave-one-strain-out CV)
4. Better than naive baselines (linear extrapolation, last-value-forward)
"""

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: Baseline Models for Comparison
# ============================================================================

class NaiveBaselines:
    """
    Simple baseline predictors to beat.
    If neural operator can't beat these, something is wrong!
    """
    
    @staticmethod
    def last_value_forward(early_data, target_length):
        """
        Predict: future = last observed value (constant extrapolation)
        """
        last_value = early_data[-1]
        prediction = np.full(target_length, last_value)
        return prediction
    
    @staticmethod
    def linear_extrapolation(early_data, early_time, target_time):
        """
        Fit line to early data, extrapolate linearly
        """
        if len(early_data) < 2:
            return np.full(len(target_time), early_data[-1])
        
        # Fit line to early data
        coeffs = np.polyfit(early_time, early_data, 1)
        slope, intercept = coeffs
        
        # Predict
        prediction = slope * target_time + intercept
        return np.maximum(prediction, 0)  # Non-negative
    
    @staticmethod
    def mean_trajectory(all_trajectories):
        """
        Predict: average trajectory across all training samples
        """
        return np.mean(all_trajectories, axis=0)


def evaluate_baselines(sequences, early_cutoff=48):
    """
    Evaluate naive baselines to set performance floor.
    """
    print("\n" + "="*70)
    print("EVALUATING BASELINE PREDICTORS")
    print("="*70)
    
    results = {
        'last_value': [],
        'linear_extrap': [],
        'mean_trajectory': []
    }
    
    # Calculate mean trajectory
    all_trajectories = [s['psilocybin'] for s in sequences]
    max_len = max(len(t) for t in all_trajectories)
    
    # Pad trajectories to same length
    padded_trajectories = []
    for traj in all_trajectories:
        if len(traj) < max_len:
            padded = np.pad(traj, (0, max_len - len(traj)), mode='edge')
        else:
            padded = traj[:max_len]
        padded_trajectories.append(padded)
    
    mean_traj = np.mean(padded_trajectories, axis=0)
    
    # Evaluate on each sequence
    for seq in sequences:
        time = seq['time']
        actual = seq['psilocybin']
        
        early_mask = time <= early_cutoff
        if early_mask.sum() < 2:
            continue
        
        early_time = time[early_mask]
        early_values = actual[early_mask]
        
        future_time = time[~early_mask]
        future_actual = actual[~early_mask]
        
        if len(future_actual) == 0:
            continue
        
        # Baseline 1: Last value forward
        pred_last = NaiveBaselines.last_value_forward(early_values, len(future_actual))
        r2_last = r2_score(future_actual, pred_last)
        results['last_value'].append(r2_last)
        
        # Baseline 2: Linear extrapolation
        pred_linear = NaiveBaselines.linear_extrapolation(early_values, early_time, future_time)
        r2_linear = r2_score(future_actual, pred_linear)
        results['linear_extrap'].append(r2_linear)
        
        # Baseline 3: Mean trajectory
        pred_mean = mean_traj[len(early_values):len(actual)]
        if len(pred_mean) > len(future_actual):
            pred_mean = pred_mean[:len(future_actual)]
        elif len(pred_mean) < len(future_actual):
            pred_mean = np.pad(pred_mean, (0, len(future_actual) - len(pred_mean)), mode='edge')
        
        r2_mean = r2_score(future_actual, pred_mean)
        results['mean_trajectory'].append(r2_mean)
    
    # Summary
    print("\nBaseline Performance (mean R² on future predictions):")
    for method, scores in results.items():
        mean_r2 = np.mean(scores)
        print(f"  {method:20s}: R² = {mean_r2:6.3f}")
    
    print("\n✓ Neural operator must beat these simple baselines!")
    
    return results


# ============================================================================
# PART 2: Hyperparameter Search
# ============================================================================

def hyperparameter_search(X_train, y_train, masks_train, 
                         X_val, y_val, masks_val,
                         param_grid=None):
    """
    Grid search over hyperparameters.
    
    Key hyperparameters:
    - hidden_dim: Model capacity
    - n_layers: Model depth
    - dropout: Regularization
    - learning_rate: Training speed
    - weight_decay: L2 regularization
    """
    
    if param_grid is None:
        param_grid = {
            'hidden_dim': [128, 256, 512],
            'dropout': [0.2, 0.3, 0.4],
            'lr': [1e-3, 5e-4, 1e-4],
            'weight_decay': [1e-4, 1e-3, 1e-2]
        }
    
    print("\n" + "="*70)
    print("HYPERPARAMETER SEARCH")
    print("="*70)
    print(f"\nSearch space:")
    for param, values in param_grid.items():
        print(f"  {param}: {values}")
    
    # Simplified search: try a few key combinations
    configs = [
        {'hidden_dim': 128, 'dropout': 0.3, 'lr': 1e-3, 'weight_decay': 1e-4},
        {'hidden_dim': 256, 'dropout': 0.3, 'lr': 5e-4, 'weight_decay': 1e-4},
        {'hidden_dim': 512, 'dropout': 0.4, 'lr': 1e-4, 'weight_decay': 1e-3},
    ]
    
    results = []
    
    for i, config in enumerate(configs):
        print(f"\n[Config {i+1}/{len(configs)}] {config}")
        
        # Build model
        from psilocybin_neural_op import TrajectoryPredictor
        
        model = TrajectoryPredictor(
            n_features=X_train.shape[1],
            max_trajectory_length=y_train.shape[1]
        )
        
        # Quick training (50 epochs)
        optimizer = Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
        
        def masked_mse_loss(pred, target, mask):
            squared_error = (pred - target) ** 2
            masked_error = squared_error * mask
            return masked_error.sum() / mask.sum()
        
        best_val_loss = float('inf')
        
        for epoch in range(50):
            model.train()
            optimizer.zero_grad()
            pred_train = model(X_train)
            loss_train = masked_mse_loss(pred_train, y_train, masks_train)
            loss_train.backward()
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                pred_val = model(X_val)
                loss_val = masked_mse_loss(pred_val, y_val, masks_val)
            
            if loss_val < best_val_loss:
                best_val_loss = loss_val
        
        # Calculate R²
        model.eval()
        with torch.no_grad():
            pred_val = model(X_val).numpy()
        
        valid_pred = pred_val[masks_val.numpy() > 0].flatten()
        valid_true = y_val.numpy()[masks_val.numpy() > 0].flatten()
        r2 = r2_score(valid_true, valid_pred)
        
        results.append({
            'config': config,
            'val_loss': best_val_loss.item(),
            'r2': r2
        })
        
        print(f"  → Val Loss: {best_val_loss:.6f}, R²: {r2:.4f}")
    
    # Find best
    best_idx = np.argmax([r['r2'] for r in results])
    best_config = results[best_idx]
    
    print("\n" + "="*70)
    print("BEST CONFIGURATION:")
    print("="*70)
    print(f"Config: {best_config['config']}")
    print(f"Val Loss: {best_config['val_loss']:.6f}")
    print(f"R²: {best_config['r2']:.4f}")
    
    return best_config, results


# ============================================================================
# PART 3: Leave-One-Strain-Out Cross-Validation
# ============================================================================

def leave_one_strain_out_cv(sequences, feature_names):
    """
    Test if model generalizes to unseen strains.
    This is the REAL test of transfer learning capability.
    """
    print("\n" + "="*70)
    print("LEAVE-ONE-STRAIN-OUT CROSS-VALIDATION")
    print("="*70)
    
    # Get unique strains
    strains = list(set(s['strain'] for s in sequences if s['strain'] is not None))
    print(f"\nTesting on {len(strains)} strains...")
    
    results = []
    
    for test_strain in strains:
        print(f"\n[Held-out strain: {test_strain}]")
        
        # Split by strain
        train_seqs = [s for s in sequences if s['strain'] != test_strain]
        test_seqs = [s for s in sequences if s['strain'] == test_strain]
        
        if len(test_seqs) == 0:
            continue
        
        print(f"  Train sequences: {len(train_seqs)}")
        print(f"  Test sequences: {len(test_seqs)}")
        
        # Prepare data (simplified - you'd use your actual preparation function)
        # This is a placeholder - integrate with your actual data prep
        print(f"  Training model...")
        
        # Would train model here on train_seqs, test on test_seqs
        # For now, just report structure
        
        results.append({
            'test_strain': test_strain,
            'n_train': len(train_seqs),
            'n_test': len(test_seqs)
        })
    
    print("\n✓ Cross-validation structure validated")
    print("  Next: Implement full training loop per fold")
    
    return results


# ============================================================================
# PART 4: Success Criteria Evaluation
# ============================================================================

def evaluate_success_criteria(model, X_val, y_val, masks_val, sequences, 
                             baseline_results, rf_r2):
    """
    Comprehensive evaluation against success criteria.
    
    SUCCESS = Model is useful for trajectory prediction
    """
    
    print("\n" + "="*80)
    print("SUCCESS CRITERIA EVALUATION")
    print("="*80)
    
    model.eval()
    with torch.no_grad():
        predictions = model(X_val).numpy()
    
    # Extract valid predictions
    valid_pred = predictions[masks_val.numpy() > 0].flatten()
    valid_true = y_val.numpy()[masks_val.numpy() > 0].flatten()
    
    # Metrics
    r2 = r2_score(valid_true, valid_pred)
    rmse = np.sqrt(mean_squared_error(valid_true, valid_pred))
    mae = np.mean(np.abs(valid_true - valid_pred))
    
    # Success criteria
    criteria = {
        'positive_r2': r2 > 0,
        'beats_last_value': r2 > np.mean(baseline_results['last_value']),
        'beats_linear': r2 > np.mean(baseline_results['linear_extrap']),
        'reasonable_rmse': rmse < 0.2,  # Less than 0.2 g/L error
        'low_mae': mae < 0.15  # Less than 0.15 g/L average error
    }
    
    print("\n📊 PERFORMANCE METRICS:")
    print(f"  R² Score:  {r2:.4f}")
    print(f"  RMSE:      {rmse:.4f} g/L")
    print(f"  MAE:       {mae:.4f} g/L")
    
    print("\n✅ SUCCESS CRITERIA:")
    passed = 0
    total = len(criteria)
    
    for criterion, passed_test in criteria.items():
        status = "✓ PASS" if passed_test else "✗ FAIL"
        print(f"  {criterion:20s}: {status}")
        if passed_test:
            passed += 1
    
    print(f"\nOverall: {passed}/{total} criteria passed")
    
    # Comparison to baselines
    print("\n📈 COMPARISON TO BASELINES:")
    baseline_means = {k: np.mean(v) for k, v in baseline_results.items()}
    
    print(f"  Neural Operator:       R² = {r2:.4f}")
    print(f"  Last Value Forward:    R² = {baseline_means['last_value']:.4f}")
    print(f"  Linear Extrapolation:  R² = {baseline_means['linear_extrap']:.4f}")
    print(f"  Mean Trajectory:       R² = {baseline_means['mean_trajectory']:.4f}")
    print(f"  Random Forest (same-time): R² = {rf_r2:.4f}")
    
    # Final verdict
    print("\n" + "="*80)
    if passed >= 3:
        print("🎉 SUCCESS! Model meets success criteria for trajectory prediction")
        print("\nNext Steps:")
        print("  1. Deploy for real-time predictions")
        print("  2. Test on new strains (transfer learning)")
        print("  3. Add uncertainty quantification")
        print("  4. Integrate into wet lab workflow")
    else:
        print("⚠️  Model needs improvement")
        print("\nRecommendations:")
        if not criteria['positive_r2']:
            print("  • R² is negative - model not learning properly")
            print("    → Check data quality, normalization, trajectory alignment")
        if not criteria['beats_linear']:
            print("  • Can't beat linear extrapolation")
            print("    → Try simpler model architecture")
            print("    → Add more training data")
        if not criteria['reasonable_rmse']:
            print("  • RMSE too high")
            print("    → Add regularization")
            print("    → Balance dataset (undersample high producers?)")
    
    return {
        'r2': r2,
        'rmse': rmse,
        'mae': mae,
        'criteria_passed': passed,
        'criteria_total': total,
        'success': passed >= 3
    }


# ============================================================================
# PART 5: Visualization of Success
# ============================================================================

def visualize_success_criteria(model, X_val, y_val, masks_val, sequences, 
                               val_idx, baseline_results, success_metrics):
    """
    Create a single "success dashboard" figure.
    """
    
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    model.eval()
    with torch.no_grad():
        predictions = model(X_val).numpy()
    
    # Panel 1: Success criteria checklist
    ax1 = fig.add_subplot(gs[0, 0])
    criteria_names = ['Positive R²', 'Beats Last-Value', 'Beats Linear', 
                     'RMSE < 0.2', 'MAE < 0.15']
    criteria_status = [1 if success_metrics['criteria_passed'] >= i else 0 
                      for i in range(1, 6)]
    
    colors = ['green' if s else 'red' for s in criteria_status]
    ax1.barh(criteria_names, [1]*len(criteria_names), color=colors, alpha=0.6, edgecolor='black')
    ax1.set_xlim([0, 1])
    ax1.set_xlabel('Pass/Fail', fontweight='bold')
    ax1.set_title('Success Criteria', fontweight='bold', fontsize=12)
    ax1.set_xticks([])
    
    # Panel 2: Baseline comparison
    ax2 = fig.add_subplot(gs[0, 1:])
    methods = ['Last Value', 'Linear Extrap', 'Mean Traj', 'Neural Operator']
    r2_scores = [
        np.mean(baseline_results['last_value']),
        np.mean(baseline_results['linear_extrap']),
        np.mean(baseline_results['mean_trajectory']),
        success_metrics['r2']
    ]
    colors = ['lightcoral', 'lightsalmon', 'lightblue', 'lightgreen']
    
    bars = ax2.bar(methods, r2_scores, color=colors, edgecolor='black', linewidth=2)
    ax2.set_ylabel('R² Score', fontweight='bold')
    ax2.set_title('Model Comparison (Trajectory Prediction)', fontweight='bold', fontsize=12)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, score in zip(bars, r2_scores):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{score:.3f}', ha='center', fontweight='bold')
    
    # Panel 3-5: Example trajectories
    example_indices = np.linspace(0, len(X_val)-1, 3, dtype=int)
    
    for i, val_seq_idx in enumerate(example_indices):
        ax = fig.add_subplot(gs[1, i])
        
        original_seq_idx = val_idx[val_seq_idx]
        seq = sequences[original_seq_idx]
        
        mask = masks_val.numpy()[val_seq_idx] > 0
        n_valid = mask.sum()
        
        actual_time = seq['time'][:n_valid]
        actual_psilocybin = seq['psilocybin'][:n_valid]
        predicted_psilocybin = predictions[val_seq_idx][:n_valid]
        
        early_cutoff = 48
        cutoff_idx = np.where(actual_time <= early_cutoff)[0][-1] if any(actual_time <= early_cutoff) else 1
        
        ax.scatter(actual_time[:cutoff_idx+1], actual_psilocybin[:cutoff_idx+1], 
                  color='green', s=60, marker='o', zorder=5, label='Early Data')
        ax.scatter(actual_time[cutoff_idx+1:], actual_psilocybin[cutoff_idx+1:], 
                  color='red', s=60, marker='s', zorder=5, label='Future')
        ax.plot(actual_time, predicted_psilocybin, 'b-', linewidth=2.5, alpha=0.7, label='Prediction')
        
        ax.set_xlabel('Time (h)', fontweight='bold')
        if i == 0:
            ax.set_ylabel('Psilocybin (g/L)', fontweight='bold')
        ax.set_title(f'{seq["strain"]}', fontweight='bold')
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
    
    # Panel 6: Predicted vs Actual
    ax6 = fig.add_subplot(gs[2, :2])
    valid_pred = predictions[masks_val.numpy() > 0].flatten()
    valid_true = y_val.numpy()[masks_val.numpy() > 0].flatten()
    
    ax6.scatter(valid_true, valid_pred, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
    max_val = max(valid_true.max(), valid_pred.max())
    ax6.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect')
    
    ax6.set_xlabel('Actual Psilocybin (g/L)', fontweight='bold')
    ax6.set_ylabel('Predicted Psilocybin (g/L)', fontweight='bold')
    ax6.set_title(f"Overall: R²={success_metrics['r2']:.3f}, RMSE={success_metrics['rmse']:.4f} g/L", 
                 fontweight='bold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    # Panel 7: Summary text
    ax7 = fig.add_subplot(gs[2, 2])
    ax7.axis('off')
    
    summary_text = f"""
SUMMARY
{'='*25}

Criteria Passed: {success_metrics['criteria_passed']}/{success_metrics['criteria_total']}

Performance:
  R² = {success_metrics['r2']:.4f}
  RMSE = {success_metrics['rmse']:.4f} g/L
  MAE = {success_metrics['mae']:.4f} g/L

Verdict:
  {'✓ SUCCESS' if success_metrics['success'] else '✗ NEEDS WORK'}

{'Model ready for deployment!' if success_metrics['success'] else 'Continue hyperparameter tuning'}
    """
    
    ax7.text(0.1, 0.5, summary_text, transform=ax7.transAxes, fontsize=10,
            verticalalignment='center', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.suptitle('Neural Operator Success Dashboard', fontsize=16, fontweight='bold', y=0.995)
    # plt.savefig('success_dashboard.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()
    print("\n✓ Success dashboard created")


# ============================================================================
# PART 6: Complete Validation Pipeline
# ============================================================================

def complete_validation_pipeline(model, X_train, y_train, masks_train,
                                X_val, y_val, masks_val, sequences, val_idx, rf_r2):
    """
    Run complete validation: baselines → hyperparameters → success criteria
    """
    
    print("\n" + "="*80)
    print("COMPLETE VALIDATION PIPELINE")
    print("="*80)
    
    # Step 1: Evaluate baselines
    baseline_results = evaluate_baselines(sequences, early_cutoff=48)
    
    # Step 2: Hyperparameter search (optional - can skip if model already trained)
    # best_config, hp_results = hyperparameter_search(X_train, y_train, masks_train,
    #                                                  X_val, y_val, masks_val)
    
    # Step 3: Evaluate success criteria
    success_metrics = evaluate_success_criteria(model, X_val, y_val, masks_val, 
                                               sequences, baseline_results, rf_r2)
    
    # Step 4: Visualize
    visualize_success_criteria(model, X_val, y_val, masks_val, sequences, 
                              val_idx, baseline_results, success_metrics)
    
    return success_metrics


if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════════╗
    ║        HYPERPARAMETER TUNING & SUCCESS VALIDATION SUITE              ║
    ╚══════════════════════════════════════════════════════════════════════╝
    
    This module provides:
    1. Baseline predictors (last-value, linear extrapolation)
    2. Hyperparameter search grid
    3. Leave-one-strain-out cross-validation
    4. Success criteria evaluation
    5. Comprehensive visualization
    
    Usage:
    ------
    from hyperparameter_tuning import complete_validation_pipeline
    
    success_metrics = complete_validation_pipeline(
        model, X_train, y_train, masks_train,
        X_val, y_val, masks_val, sequences, val_idx, rf_r2
    )
    
    if success_metrics['success']:
        print("🎉 Model validated - ready for deployment!")
    else:
        print("Continue tuning...")
    """)


    ╔══════════════════════════════════════════════════════════════════════╗
    ║        HYPERPARAMETER TUNING & SUCCESS VALIDATION SUITE              ║
    ╚══════════════════════════════════════════════════════════════════════╝

    This module provides:
    1. Baseline predictors (last-value, linear extrapolation)
    2. Hyperparameter search grid
    3. Leave-one-strain-out cross-validation
    4. Success criteria evaluation
    5. Comprehensive visualization

    Usage:
    ------
    from hyperparameter_tuning import complete_validation_pipeline

    success_metrics = complete_validation_pipeline(
        model, X_train, y_train, masks_train,
        X_val, y_val, masks_val, sequences, val_idx, rf_r2
    )

    if success_metrics['success']:
        print("🎉 Model validated - ready for deployment!")
    else:
        print("Continue tuning...")
    
